# Введение
В данном ноутбуке собраны все материалы из главы 1.4 OpenStreetMap из курса "Геопространственный анализ данных". Рассматриваются методы загрузки данных с OpenStreetMap

In [ ]:
!pip -q install overpass overpy

# OverPass API

In [ ]:
# Необходимые импорты
import numpy as np
import requests
import overpass
import overpy
from requests.exceptions import RequestException, Timeout

## Через request

Пример запросов напрямую через `requests`

В качестве запроса будем находить количество заведений типа кафе вокруг 1500м от точки с координатами 56.15962, 38.87188, которая находится в городе Киржач, Владимирской области.

In [ ]:
# Необходимые импорты
import numpy as np
import requests
from requests.exceptions import RequestException, Timeout

def get_count_cafe_around_point(latitude, longitude, radius):

    # URL сервисов
    # overpass_url = "https://overpass-api.de/api/interpreter"
    overpass_url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"

    # Формируем запрос overpass
    # вывод в формате json
    # ищем все точки, линии, отношения, у которых есть тег "amenity"="cafe"
    # в радиусе radius от точки с координатами (latitude,longitude)
    # в выводе просим указать количество
    overpass_query = f"""
    [out:json][timeout:25];
    nwr["amenity"="cafe"](around:{radius},{latitude},{longitude});
    out count;
    """

    try:
        # отправляем запрос
        response = requests.get(overpass_url, params={'data': overpass_query})
        response.raise_for_status()
        data = response.json()
        # выбираем нужное значение
        if 'elements' in data:
            for element in data['elements']:
                if 'tags' in element:
                    for key, value in element['tags'].items():
                        if key == 'total':
                            return value

        # Если нет тега total, или вообще нет тегов
        return np.NaN

    # обработки ошибок
    except RequestException as e:
        print(f"RequestException: {e}")
        return np.NaN
    except Timeout:
        print("Request timed out.")
        return np.NaN
    except Exception as e:
        print(f"An error occurred: {e}")
        return np.NaN

# Пример
coords = (56.15962, 38.87188) # город Киржач
radius = 1500 # в метрах

result = get_count_cafe_around_point(coords[0], coords[1], radius)
print(f'В радиусе {radius} м от точки {coords} находится {result} кафе')

В радиусе 1500 м от точки (56.15962, 38.87188) находится 8 кафе


## Через overpy

Существуют различные обертки для Overpass API, попробуем парочку из них

In [ ]:
import overpy

# Аналогично запросу через  requests, сначала указываем куда слать запрос
overpass_url = "https://maps.mail.ru/osm/tools/overpass/api/interpreter"
api = overpy.Overpass(url=overpass_url)

# Затем сам запрос пишем
result = api.query("""
    [out:json][timeout:25];
    nwr["amenity"="cafe"](around:1500, 56.15962, 38.87188);
    out;
    """)

# данная обертка не умеет выводить общее количество, поэтому руками считаем
# общее количество
count_result = len(result.ways) + len(result.nodes) + len(result.areas)
print(f'В радиусе 1500 м от точки 56.15962, 38.87188 находится {count_result} кафе')
# строка разделитель
print('***'*20)

# Дополнительно вывыдем всю возможную информацию с помощью данной обертки
for point in result.nodes:
    print(f'Point ID={point.id}, tags={point.tags}, lat={point.lat}, lon={point.lon}')

В радиусе 1500 м от точки 56.15962, 38.87188 находится 8 кафе
************************************************************
Point ID=2701046998, tags={'amenity': 'cafe', 'name': 'Аида'}, lat=56.1615826, lon=38.8728146
Point ID=6690633237, tags={'amenity': 'cafe', 'contact:email': 'horeca@33pingvina.ru', 'contact:phone': '+7 800 1005033', 'contact:website': 'https://33pingvina.ru/', 'cuisine': 'ice_cream', 'name': '33 пингвина', 'opening_hours': 'Mo-Su 09:00-19:00', 'operator': 'Интеропт', 'shop': 'ice_cream'}, lat=56.1624110, lon=38.8716120
Point ID=6743529274, tags={'amenity': 'cafe', 'name': 'Лакомка'}, lat=56.1525794, lon=38.8518852
Point ID=6798201569, tags={'amenity': 'cafe', 'contact:phone': '+7 916 5780664', 'name': 'Столовая', 'opening_hours': 'Mo-Su 09:00-20:00'}, lat=56.1630449, lon=38.8713751
Point ID=10757580248, tags={'amenity': 'cafe', 'contact:phone': '+7 49237 29602', 'contact:vk': 'https://vk.com/diana_official_page', 'contact:vk2': 'https://vk.com/market-66180628', 'na

## Через overpass

In [ ]:
# импортируем другую обертку
import overpass
# указыавем куда слать запросы
api = overpass.API(endpoint=overpass_url)

# сам запрос, обратите внимание в данном инструменте, формат вывода указывается
# через responseformat, а результаты вывода можно указать через verbosity
response = api.get('nwr["amenity"="cafe"](around:1500, 56.15962, 38.87188)',
                   responseformat="json",
                   verbosity='count')
# Выводится аналогично получению ответа через requests
response

{'version': 0.6,
 'generator': 'Overpass API 0.7.57.1 74a55df1',
 'osm3s': {'timestamp_osm_base': '2024-03-24T13:08:25Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'count',
   'id': 0,
   'tags': {'nodes': '7', 'ways': '1', 'relations': '0', 'total': '8'}}]}

In [ ]:
# поэтому выбираем нужное нам значение
if 'elements' in response:
    for element in response['elements']:
        if 'tags' in element:
            for key, value in element['tags'].items():
                if key == 'total':
                    result = value
print(f'В радиусе 1500 м от точки 56.15962, 38.87188 находится {result} кафе')

В радиусе 1500 м от точки 56.15962, 38.87188 находится 8 кафе


# OSMnx

Установка и импорт библиотеки

In [ ]:
# Установка и импорт библиотеки
!pip -q install osmnx
import osmnx as ox

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.3/104.3 kB 2.1 MB/s eta 0:00:00


In [ ]:
# город Киржач
coords = (56.15962, 38.87188)
# Теги, которые нам нужны, если нужна вся категория, то можно передать
# 'amenity':True
tags = {'amenity':['cafe']}
dist = 1500 # в метрах

features = ox.features.features_from_point(coords, tags, radius)
result = len(features)
print(f'В пределах квадрата со стороной {dist*2} м с центром в точке {coords} находится {result} кафе')

В пределах квадрата со стороной 3000 м с центром в точке (56.15962, 38.87188) находится 10 кафе


In [ ]:
features.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bo

amenity   contact:phone                contact:vk  \
element_type osmid                                                          
node         1827315019    cafe  +7 960 7224606  https://vk.com/cafeglobe   
             2701046998    cafe             NaN                       NaN   
             6690633237    cafe  +7 800 1005033                       NaN   
             6743529274    cafe             NaN                       NaN   
             6798201569    cafe  +7 916 5780664                       NaN   

                                contact:website delivery         name  \
element_type osmid                                                      
node         1827315019        www.kafe-glob.ru      yes         Глоб   
             2701046998                     NaN      NaN         Аида   
             6690633237  https://33pingvina.ru/      NaN  33 пингвина   
             6743529274                     NaN      NaN      Лакомка   
             6798201569                     NaN      NaN     Столовая   

                             opening_hours                   geometry  \
element_type osmid                                                      
node         1827315019  Mo-Su 10:00-24:00  POINT (38.84813 56.14846)   
             2701046998                NaN  POINT (38.87281 56.16158)   
             6690633237  Mo-Su 09:00-19:00  POINT (38.87161 56.16241)   
             6743529274                NaN  POINT (38.85189 56.15258)   
             6798201569  Mo-Su 09:00-20:00  POINT (38.87138 56.16304)   

                                contact:email    cuisine  operator       shop  \
element_type osmid                                                              
node         1827315019                   NaN        NaN       NaN        NaN   
             2701046998                   NaN        NaN       NaN        NaN   
             6690633237  horeca@33pingvina.ru  ice_cream  Интеропт  ice_cream   
             6743529274                   NaN        NaN       NaN        NaN   
             6798201569                   NaN        NaN       NaN        NaN   

                        contact:vk2 takeaway nodes addr:housenumber  \
element_type osmid                                                    
node         1827315019         NaN      NaN   NaN              NaN   
             2701046998         NaN      NaN   NaN              NaN   
             6690633237         NaN      NaN   NaN              NaN   
             6743529274         NaN      NaN   NaN              NaN   
             6798201569         NaN      NaN   NaN              NaN   

                        addr:street building  
element_type osmid                            
node         1827315019         NaN      NaN  
             2701046998         NaN      NaN  
             6690633237         NaN      NaN  
             6743529274         NaN      NaN  
             6798201569         NaN      NaN

# PyrOSM

In [ ]:
# Установка библиотеки
!pip -q install pyrosm

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 9.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# скачаем дамп карты OpenStreetMap для ЦФО
!gdown https://download.geofabrik.de/russia/central-fed-district-latest.osm.pbf

Downloading...
From: https://download.geofabrik.de/russia/central-fed-district-latest.osm.pbf
To: /content/central-fed-district-latest.osm.pbf
100% 781M/781M [00:40<00:00, 19.3MB/s]


Установим аналог osmosis - [osmium](https://osmcode.org/osmium-tool/manual.html). Этот инструмент необходим, для того, чтобы вырезать Владимирскую область из дампа ЦФО. Так как pyrosm плохо работает с большими файлами (более 100мб).

In [ ]:
!apt-get -qq install osmium-tool

Selecting previously unselected package osmium-tool.
(Reading database ... 121753 files and directories currently installed.)
Preparing to unpack .../osmium-tool_1.14.0-1_amd64.deb ...
Unpacking osmium-tool (1.14.0-1) ...
Setting up osmium-tool (1.14.0-1) ...
Processing triggers for man-db (2.10.2-1) ...


In [ ]:
# первой командой вырезаем границы Владимирской области и сохраняем в файл.
# r72197 - id области в OSM, найти можно по поиску на сайте openstreetmap.org
!osmium getid -r central-fed-district-latest.osm.pbf r72197 -o vladimirskaya_obl_boundary.osm

[======================================================================] 100% 


In [ ]:
# второй командой вырезаем всю Владимирскую область из ЦФО и сохраняем в файл
!osmium extract -p vladimirskaya_obl_boundary.osm central-fed-district-latest.osm.pbf -o vladimirskaya_obl.osm.pbf

[======================================================================] 100% 


Так как у данной библиотеки нет метода для поиска нужных точек вокруг точки с координатами, поэтому придется воспользоваться костылем.

In [ ]:
# импортируем нужные библиотеки
import geopandas as gpd
from shapely import Point

# Создаем полигон из точки
coords = (56.15962, 38.87188)
dist = 1500
point = gpd.GeoDataFrame({'name': ['point'],
                           'geometry': [Point(coords[1], coords[0])]},
                          crs="EPSG:4326")
polygon_from_point = point.to_crs(epsg=32637).buffer(dist).to_crs(epsg=4326)[0]

/usr/local/lib/python3.10/dist-packages/pyproj/transformer.py:820: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return self._transformer._transform_point(


In [ ]:
# импортируем библиотеку
import pyrosm
# Считываем данные и фильтруем по полигону
osm = pyrosm.OSM('/content/vladimirskaya_obl.osm.pbf', bounding_box=polygon_from_point)
custom_filter = {'amenity': ['cafe']}
# находим все кафе
pois = osm.get_pois(custom_filter=custom_filter)
pois

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<frozen importlib._bootstrap>:914: ImportWarning: _PyDrive2ImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _GenerativeAIImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: _OpenCVImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:914: ImportWarning: APICoreClientInfoImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bo

,changeset,visible,timestamp,lat,tags,lon,id,version,name,opening_hours,operator,amenity,geometry,osm_type,addr:housenumber,addr:street,building
0,0.0,False,1698121134,56.161583,None,38.872814,2701046998,17,Аида,None,None,cafe,POINT (38.87281 56.16158),node,NaN,NaN,NaN
1,0.0,False,1698121134,56.162411,"{""contact:email"":""horeca@33pingvina.ru"",""conta...",38.871613,6690633237,26,33 пингвина,Mo-Su 09:00-19:00,Интеропт,cafe,POINT (38.87161 56.16241),node,NaN,NaN,NaN
2,0.0,False,1698121134,56.152580,None,38.851887,6743529274,17,Лакомка,None,None,cafe,POINT (38.85189 56.15258),node,NaN,NaN,NaN
3,0.0,False,1698121134,56.163044,"{""contact:phone"":""+7 916 5780664""}",38.871376,6798201569,19,Столовая,Mo-Su 09:00-20:00,None,cafe,POINT (38.87138 56.16304),node,NaN,NaN,NaN
4,0.0,False,1698121134,56.153770,"{""contact:phone"":""+7 49237 29602"",""contact:vk""...",38.854229,10757580248,17,Диана,Mo-Su 10:00-22:00,None,cafe,POINT (38.85423 56.15377),node,NaN,NaN,NaN
5,0.0,False,1698121134,56.171970,"{""contact:phone"":""+7 965 2292000"",""takeaway"":""...",38.866249,11170704998,5,Адмирал,None,None,cafe,POINT (38.86625 56.17197),node,NaN,NaN,NaN
6,0.0,False,1700123770,56.161953,"{""contact:phone"":""+7 960 7330055""}",38.873600,11353602993,1,Подвальчик,Mo-Th 09:00-18:00; Fr-Su 09:00-20:00,None,cafe,POINT (38.87360 56.16195),node,NaN,NaN,NaN
7,NaN,False,1698121134,NaN,"{""contact:phone"":""+7 919 0022080""}",NaN,572750220,19,Caramel,"Mo-Fr 09:00-24:00; Sa,Su 00:00-02:00,09:00-24:00",NaN,cafe,"POLYGON ((38.87139 56.15982, 38.87157 56.15993...",way,17А,Ленинградская улица,yes


А на этом все!